# 如何处理工具错误

:::info 先决条件

本指南假设备您熟悉以下概念：
- [Chat models](/docs/concepts/chat_models)
- [LangChain Tools](/docs/concepts/tools)
- [如何使用模型调用工具](/docs/how_to/tool_calling)

:::

使用 LLM [调用工具](/docs/concepts/tool_calling/) 通常比纯提示更可靠，但并非完美无缺。模型可能会尝试调用一个不存在的工具，或者无法返回与请求的 schema 匹配的参数。采用使 schema 保持简单、减少一次传递的工具数量，以及提供良好的名称和描述等策略可以帮助降低这种风险，但并非万无一失。

本指南将介绍在 chains 中构建错误处理的一些方法，以缓解这些故障模式。

## 设置

我们需要安装以下包：

In [ ]:
%pip install --upgrade --quiet langchain-core langchain-openai

如果你想在 [LangSmith](https://docs.smith.langchain.com/) 中追踪你的运行，请取消注释并设置以下环境变量：

In [2]:
import getpass
import os

# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## Chain

假设我们有以下（模拟的）工具和工具调用链。我们将故意让我们的工具变得复杂，以试图让模型出错。

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm"/>

In [3]:
# | echo: false
# | output: false

from langchain_openai import ChatOpenAI

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [4]:
# Define tool
from langchain_core.tools import tool


@tool
def complex_tool(int_arg: int, float_arg: float, dict_arg: dict) -> int:
    """Do something complex with a complex tool."""
    return int_arg * float_arg


llm_with_tools = llm.bind_tools(
    [complex_tool],
)

# Define chain
chain = llm_with_tools | (lambda msg: msg.tool_calls[0]["args"]) | complex_tool

我们可以看到，当我们尝试使用一个相当明确的输入来调用这个链时，模型未能正确调用工具（它忘记了 `dict_arg` 参数）。

In [5]:
chain.invoke(
    "use complex tool. the args are 5, 2.1, empty dictionary. don't forget dict_arg"
)

ValidationError: 1 validation error for complex_toolSchema
dict_arg
  Field required [type=missing, input_value={'int_arg': 5, 'float_arg': 2.1}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing

## 尝试/异常工具调用

更优雅地处理错误的简单方法是尝试/异常工具调用步骤，并在出错时返回一条有用的消息：

In [6]:
from typing import Any

from langchain_core.runnables import Runnable, RunnableConfig


def try_except_tool(tool_args: dict, config: RunnableConfig) -> Runnable:
    try:
        complex_tool.invoke(tool_args, config=config)
    except Exception as e:
        return f"Calling tool with arguments:\n\n{tool_args}\n\nraised the following error:\n\n{type(e)}: {e}"


chain = llm_with_tools | (lambda msg: msg.tool_calls[0]["args"]) | try_except_tool

print(
    chain.invoke(
        "use complex tool. the args are 5, 2.1, empty dictionary. don't forget dict_arg"
    )
)

Calling tool with arguments:

{'int_arg': 5, 'float_arg': 2.1}

raised the following error:

<class 'pydantic_core._pydantic_core.ValidationError'>: 1 validation error for complex_toolSchema
dict_arg
  Field required [type=missing, input_value={'int_arg': 5, 'float_arg': 2.1}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing


## 后备方案

在工具调用出错时，我们还可以尝试回退到更好的模型。在这种情况下，我们将回退到使用 `gpt-4-1106-preview` 而不是 `gpt-3.5-turbo` 的相同链。

In [7]:
chain = llm_with_tools | (lambda msg: msg.tool_calls[0]["args"]) | complex_tool

better_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0).bind_tools(
    [complex_tool], tool_choice="complex_tool"
)

better_chain = better_model | (lambda msg: msg.tool_calls[0]["args"]) | complex_tool

chain_with_fallback = chain.with_fallbacks([better_chain])

chain_with_fallback.invoke(
    "use complex tool. the args are 5, 2.1, empty dictionary. don't forget dict_arg"
)

10.5

查看此链运行的 [LangSmith trace](https://smith.langchain.com/public/00e91fc2-e1a4-4b0f-a82e-e6b3119d196c/r)，我们可以看到第一次链调用如预期失败，而回退成功了。

## 重试与异常

为了进一步完善，我们可以尝试将异常传入并自动重新运行链，以便模型能够纠正其行为：

In [8]:
from langchain_core.messages import AIMessage, HumanMessage, ToolCall, ToolMessage
from langchain_core.prompts import ChatPromptTemplate


class CustomToolException(Exception):
    """Custom LangChain tool exception."""

    def __init__(self, tool_call: ToolCall, exception: Exception) -> None:
        super().__init__()
        self.tool_call = tool_call
        self.exception = exception


def tool_custom_exception(msg: AIMessage, config: RunnableConfig) -> Runnable:
    try:
        return complex_tool.invoke(msg.tool_calls[0]["args"], config=config)
    except Exception as e:
        raise CustomToolException(msg.tool_calls[0], e)


def exception_to_messages(inputs: dict) -> dict:
    exception = inputs.pop("exception")

    # Add historical messages to the original input, so the model knows that it made a mistake with the last tool call.
    messages = [
        AIMessage(content="", tool_calls=[exception.tool_call]),
        ToolMessage(
            tool_call_id=exception.tool_call["id"], content=str(exception.exception)
        ),
        HumanMessage(
            content="The last tool call raised an exception. Try calling the tool again with corrected arguments. Do not repeat mistakes."
        ),
    ]
    inputs["last_output"] = messages
    return inputs


# We add a last_output MessagesPlaceholder to our prompt which if not passed in doesn't
# affect the prompt at all, but gives us the option to insert an arbitrary list of Messages
# into the prompt if needed. We'll use this on retries to insert the error message.
prompt = ChatPromptTemplate.from_messages(
    [("human", "{input}"), ("placeholder", "{last_output}")]
)
chain = prompt | llm_with_tools | tool_custom_exception

# If the initial chain call fails, we rerun it withe the exception passed in as a message.
self_correcting_chain = chain.with_fallbacks(
    [exception_to_messages | chain], exception_key="exception"
)

In [9]:
self_correcting_chain.invoke(
    {
        "input": "use complex tool. the args are 5, 2.1, empty dictionary. don't forget dict_arg"
    }
)

10.5

我们的链成功了！查看 [LangSmith trace](https://smith.langchain.com/public/c11e804c-e14f-4059-bd09-64766f999c14/r)，我们可以看到我们的初始链确实仍然失败，只有在重试后链才成功。

## 后续步骤

您已经了解了几种处理工具调用错误的方法。接下来，您可以了解更多关于如何使用工具的信息：

- 示例提示（Few shot prompting）[结合工具](/docs/how_to/tools_few_shot/)
- 流式传输 [工具调用](/docs/how_to/tool_streaming/)
- 将 [运行时值传递给工具](/docs/how_to/tool_runtime)

您还可以查看一些更具体的工具调用用法：

- 从模型获取[结构化输出](/docs/how_to/structured_output/)